In [1]:
import warnings
warnings.simplefilter(action='ignore')
import nltk
import pandas as pd

In [2]:
train_data = pd.read_csv(r"D:\dev0608\treino.csv", delimiter=';')
test_data = pd.read_csv(r"D:\dev0608\avaliacao.csv", delimiter=';')

In [3]:
train_data

,cod_treino,titulo,categoria,cor,tamanho,marca,tipo,referencia,voltagem,ambiente,subcategoria,linha
0,1,"Piso Laminado Click Nature 136x21,7cm m² Artens",Piso Laminado,Nature,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,Pisos,Click
1,2,"Piso Laminado Click Elmo Ravena 136x21,7cm m² ...",Piso Laminado,Ravena,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,Pisos,Click
2,3,"Piso Laminado Click Legno Blanche 136x21,7cm m...",Piso Laminado,Blanche,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,Pisos,Click
3,4,"Piso Laminado Click Evidence Kalahari 136x21,7...",Piso Laminado,Bege,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,Pisos,Click
4,5,"Piso Laminado Click Spot Carvalho 134x18,7cm m...",Piso Laminado,Carvalho,"134x18,7",Eucafloor,Pisos,NaN,NaN,Interno,Pisos,Click
...,...,...,...,...,...,...,...,...,...,...,...,...
371,14,Guarda-Roupa Casal Ana 6 PT 4 GV Branco 240 cm,Guarda-roupas Casal,Branco,NaN,NaN,Moveis de casal,NaN,NaN,Quarto,Moveis para Quarto,NaN
372,15,Sofá 3 Lugares Living Algodão e Poliéster Cinz...,Sofas,Cinza,NaN,Living,Moveis Estofados,NaN,NaN,Sala & Escritorio,Moveis para Quarto,NaN
373,16,Sofá 4 Lugares Living Poliéster Bege,Sofas,Bege,NaN,Living,Moveis Estofados,NaN,NaN,Sala & Escritorio,Moveis para Quarto,NaN
374,17,Mesa de Jantar Retangular Paraty Amêndoa e Pre...,NaN,Amêndoa & Perola,NaN,NaN,Moveis para cozinha,NaN,NaN,Cozinha,Moveis para Cozinha e Sala de estar,NaN


In [4]:
train_data.columns

Index(['cod_treino', 'titulo', 'categoria', 'cor', 'tamanho', 'marca', 'tipo',
       'referencia', 'voltagem', 'ambiente', 'subcategoria', 'linha'],
      dtype='object')

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer

def prepare_data(data: pd.DataFrame) -> DataLoader:
    # Carregar o tokenizer
    tokenizer = AutoTokenizer.from_pretrained("nome_do_modelo")

    # Codificar os títulos usando o tokenizer
    train_encodings = tokenizer(list(data['titulo']), truncation=True, padding=True, max_length=128)

    # Obter rótulos da coluna "categoria"
    train_labels = data['categoria']

    # Converter rótulos em números
    label_to_id = {label: idx for idx, label in enumerate(train_labels.unique())}
    train_labels = train_labels.map(label_to_id)

    # Criar o conjunto de dados usando TensorDataset
    train_dataset = TensorDataset(
        torch.tensor(train_encodings['input_ids']),
        torch.tensor(train_encodings['attention_mask']),
        torch.tensor(train_labels.tolist())
    )

    # Criar o DataLoader
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

    return train_loader

# Carregar o conjunto de treinamento
treino = pd.read_csv("caminho_do_seu_arquivo.csv")

# Preparar o conjunto de treinamento
train_loader = prepare_data(treino)

# Exibir um lote de dados
for batch in train_loader:
    input_ids, attention_mask, labels = batch
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
    break  # Exibindo apenas o primeiro lote para ilustração


In [3]:
from typing import List, Dict, Tuple
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.optim.lr_scheduler import OneCycleLR
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def load_data(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path, sep=";")

def prepare_data(data: pd.DataFrame) -> Tuple[DataLoader, DistilBertTokenizer]:
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
    train_encodings = tokenizer(list(data['titulo']), truncation=True, padding=True, max_length=128)
    train_labels = data['categoria']
    label_to_id = {label: idx for idx, label in enumerate(train_labels.unique())}
    train_labels = train_labels.map(label_to_id)
    train_dataset = TensorDataset(
        torch.tensor(train_encodings['input_ids']),
        torch.tensor(train_encodings['attention_mask']),
        torch.tensor(train_labels.tolist())
    )
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
    return train_loader, tokenizer


def train_model(train_loader: DataLoader, num_labels: int) -> DistilBertForSequenceClassification:
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = OneCycleLR(optimizer, max_lr=5e-5, total_steps=len(train_loader) * 5)
    loss_fn = torch.nn.CrossEntropyLoss()
    for epoch in range(5):
        model.train()
        for step, batch in enumerate(train_loader):
            optimizer.zero_grad()
            inputs, masks, labels = batch
            outputs = model(inputs, attention_mask=masks).logits
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
    return model


def match_info(treino: pd.DataFrame, avaliacao: pd.DataFrame, model: DistilBertForSequenceClassification, label_encoder: LabelEncoder, tokenizer: DistilBertTokenizer) -> List[Dict]:
    matching_info = []
    for index, avaliacao_row in avaliacao.iterrows():
        titulo_avaliacao = avaliacao_row['titulo']
        matching_rows = treino[treino['titulo'] == titulo_avaliacao]
        if not matching_rows.empty:
            matching_row = matching_rows.iloc[0]
            categoria_treino = matching_row['categoria']
            input_text = titulo_avaliacao
            input_encoding = tokenizer(input_text, truncation=True, padding=True, max_length=128)
            input_ids = torch.tensor(input_encoding['input_ids']).unsqueeze(0)
            attention_mask = torch.tensor(input_encoding['attention_mask']).unsqueeze(0)
            predicted_label = torch.argmax(model(input_ids, attention_mask=attention_mask).logits, dim=1).item()
            predicted_category = label_encoder.inverse_transform([predicted_label])[0]
            matching_info.append({
                'titulo_avaliacao': titulo_avaliacao,
                'categoria_treino': categoria_treino,
                'categoria_prevista': predicted_category
            })
    return matching_info



def main() -> None:
    treino = load_data(r"D:\dev0608\treino.csv")
    avaliacao = load_data(r"D:\dev0608\avaliacao.csv")
    train_loader, tokenizer = prepare_data(treino)
    num_labels = len(treino['categoria'].unique())
    model = train_model(train_loader, num_labels)
    label_encoder = LabelEncoder()
    label_encoder.fit(treino['categoria'])
    matching_info = match_info(treino, avaliacao, model, label_encoder, tokenizer)
    for info in matching_info:
        print(info)

if __name__ == "__main__":
    main()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\dev0608\venv\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
import pandas as pd

# Carregar os dados
df = pd.read_csv(r"D:\dev0608\excelfiles\obras0308.csv", sep=";", encoding="latin-1")
df.fillna(0, inplace=True)
df.columns

Index(['ï»¿Badges-Quantidadedeobras', 'Nome', 'Logradouro', 'Bairro', 'CEP',
       'Municipio', 'UF', 'ValorTotalObras', 'MetragemTotal', 'obrasporano',
       'quantidadedeobras', 'duracamediadasobras', 'cidadepreferida',
       'bairropreferido', 'municipio', 'bairroobra', 'municipioobra',
       'obrasuf', 'metoquadradobrafaixa'],
      dtype='object')

In [ ]:
import pandas as pd

# Carregar os dados
df = pd.read_csv(r"D:\dev0608\excelfiles\obras0308.csv", sep=";", encoding="latin-1")
df.fillna(0, inplace=True)

# Função para verificar e converter os valores da coluna 'metoquadradobrafaixa'
def verificar_e_converter(valor):
    if isinstance(valor, str):
        return valor
    elif isinstance(valor, int):
        if valor == 0:
            return 'Não especificado'
        elif valor > 500:
            return 'acima de 500'
        elif valor < 100:
            return 'abaixo de 100'
        elif valor >= 300 and valor <= 500:
            return '300 a 500'
        elif valor >= 200 and valor < 300:
            return '200 a 300'
        elif valor >= 100 and valor < 200:
            return '100 a 200'
    return 'Não especificado'


df['metoquadradobrafaixa'] = df['metoquadradobrafaixa'].apply(verificar_e_converter)

# Agrupar por UF e realizar as operações de agregação
result = df.groupby('UF').agg(
    Quantidade_Nome=('ï»¿Badges-Quantidadedeobras', 'count'),
    Soma_ValorTotalObras=('ValorTotalObras', 'sum'),
    Soma_MetragemTotal=('MetragemTotal', 'sum'),
    Soma_obrasporano=('obrasporano', 'sum'),
    Soma_quantidadedeobras=('quantidadedeobras', 'sum'),
    Soma_duracamediadasobras=('duracamediadasobras', 'sum'),
    Contagem_cidadepreferida=('cidadepreferida', 'sum'),
    Contagem_bairropreferido=('bairropreferido', 'sum'),
    Contagem_municipio=('municipio', 'sum'),
    Contagem_bairroobra=('bairroobra', 'sum'),
    Contagem_municipioobra=('municipioobra', 'sum'),
    Contagem_obrasuf=('obrasuf', 'sum'),
    Contagem_metoquadradobrafaixa=('metoquadradobrafaixa', 'count')
).reset_index()

print(result)


